In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

FHIR_DIR = "/content/drive/MyDrive/Patient_records/fhir (FHIR R4 Synthea)"
print("FHIR bundles will be read from:", FHIR_DIR)

Mounted at /content/drive
FHIR bundles will be read from: /content/drive/MyDrive/Patient_records/fhir (FHIR R4 Synthea)


In [2]:
%%bash
# (re‑)create venv
python -m pip install --quiet virtualenv
python -m virtualenv /content/fhir_env

# activate & upgrade pip
source /content/fhir_env/bin/activate
pip install --quiet --upgrade pip

# core libs (NumPy<2 for compatibility, PyTorch 2.3 CUDA 12, etc.)
pip install --quiet "numpy<2" \
    --extra-index-url https://download.pytorch.org/whl/cu121 \
    torch==2.3.0 torchvision==0.18.0

# helper libs for the clinical assistant
pip install --quiet \
    fhir.resources==7.1.0 \
    sentence-transformers==2.7.0 \
    transformers==4.43.2 accelerate==0.30.1 bitsandbytes==0.43.0 \
    langchain==0.2.0 langchain-community==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 38.0 MB/s eta 0:00:00
created virtual environment CPython3.11.13.final.0-64 in 339ms
  creator CPython3Posix(dest=/content/fhir_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 141.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 133.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 136.7 MB/s et

In [3]:
%%bash
rm -rf /content/faiss
rm -rf /content/fhir_env/lib/python3.11/site-packages/faiss*
rm -rf /content/fhir_env/lib/python3.11/site-packages/_swigfaiss*

In [4]:
%%bash
source /content/fhir_env/bin/activate
pip install --upgrade --force-reinstall "numpy<2"

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [5]:
%%bash
source /content/fhir_env/bin/activate
sudo apt-get update -qq
sudo apt-get install -y --no-install-recommends \
     swig python3-dev build-essential cmake git libopenblas-dev libomp-dev

rm -rf faiss
git clone --branch v1.7.4 https://github.com/facebookresearch/faiss.git
cd faiss
cmake -B build -DFAISS_ENABLE_GPU=ON -DFAISS_ENABLE_PYTHON=ON \
               -DFAISS_ENABLE_C_API=OFF -DCMAKE_BUILD_TYPE=Release \
               -DPython_EXECUTABLE=$(which python)
cmake --build build --target faiss faiss_avx2 swigfaiss -j $(nproc)
cd build/faiss/python
pip install .

Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.9ubuntu3).
libopenblas-dev is already the newest version (0.3.20+ds-1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.15).
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
The following additional packages will be installed:
  libomp-14-dev libomp5-14 swig4.0
Suggested packages:
  libomp-14-doc swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14 swig swig4.0
0 upgraded, 5 newly installed, 0 to remove and 36 not upgraded.
Need to get 1,854 kB of archives.
After this operation, 14.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Cloning into 'faiss'...
Note: switching to 'd87888b13e7eb339bb9c45825e9d20def6665171'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you c

In [8]:
import sys
sys.path.insert(0, "/content/faiss/build/faiss/python")
import faiss
print("faiss imported", faiss.__version__)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [9]:
import faiss, torch, numpy as np
d = 32
cpu_index = faiss.IndexFlatL2(d)
res       = faiss.StandardGpuResources()
gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)

xb = np.random.randn(10_000, d).astype('float32')
gpu_index.add(xb)
D, _ = gpu_index.search(xb[:5], 3)
print("✓ GPU Faiss operational – distances shape:", D.shape)

print("CUDA visible to Torch?:", torch.cuda.is_available(),
      "| device name:", torch.cuda.get_device_name(0))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import